In [1]:
from searchPlus import *
from collections import namedtuple


line1 = "## ## ## ## ## ## ## ## ##\n"
line2 = "## .. .. .. .. .. .. .. ##\n"
line3 = "## .. .. .. 00 .. .. .. ##\n"
line4 = "## .. .. .. .. .. .. .. ##\n"
line5 = "## .. .. () () () .. .. ##\n"
line6 = "## .. .. .. .. .. .. .. ##\n"
line7 = "## .. .. .. 01 .. .. .. ##\n"
line8 = "## .. .. .. .. .. .. .. ##\n"
line9 = "## ## ## ## ## ## ## ## ##\n"
grid = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8 + line9


EstadoPin = namedtuple('EstadoPin','pinguins')

class EstadoPinguins(EstadoPin):
        
    def __hash__(self):
        return hash(tuple(sorted(self.pinguins.items())))
    
    def __lt__(self,other):
        return True
    
    
class PenguinsPairs(Problem):
    
    def process_txt(self, grid):
        """
        Função que porcessa a grelha em txt e obtém as posições dos icebergues (walls), água e pinguins.
        O output desta função é um dicionário em que as chaves identifica cada tipo de informação a ser 
        guardada. Os pinguins serão um dicionário e os icebergues e água serão um conjunto.
        """
        data = {'walls': set(), 'pinguins': {}, 'water': set()}
        lines = grid.split('\n')[:-1]
        x = 0
        for row in lines:
            ll = row.split()
            y = 0
            for col in ll:
                if col == '##':
                    data['walls'].add((x,y))
                elif col == '()':
                    data['water'].add((x,y))
                elif col != '..':
                    data['pinguins'][col] = (x,y)
                y += 1
            x += 1
        data['dim'] = (len(lines),len(ll))
        return data
    
    
    directions = {"N":(-1, 0), "E":(0, +1), "S":(+1, 0), "O":(0, -1)}  # ortogonais
    
    
    def __init__(self, ice_map=grid):
        initialStatus = self.process_txt(ice_map) # processa o txt e converte num dicionário
        self.initial = EstadoPinguins(initialStatus['pinguins']) # estado inicial
        self.goal = {} # estado vazio
        self.obstacles = initialStatus['walls'] # posições do icebergues
        self.water = initialStatus['water'] # posições da água
        self.dim = initialStatus['dim'] # dimensão do mapa (não precisa de ser quadrado)

    
    def slide(self,state,x,y,d):
        """
        Função que identifica se é possível o pinguim se deslocar numa direção.
        """
        (dx,dy) = self.directions[d]
        if (x+dx,y+dy) in self.obstacles:
            return False
        while (x+dx,y+dy) not in self.obstacles and (x+dx,y+dy) not in state.pinguins.values():
            if (x+dx,y+dy) in self.water:
                return False
            x = x+dx
            y = y+dy
        return True
    
    
    def actions (self, state):
        """
        Devolve uma lista ordenada com todas as ações possíveis para o estado.
        """
        actions_list = []
        for p in state.pinguins.keys():
            x,y = state.pinguins[p] # coordenadas da posição do pinguim 
            if self.slide(state,x,y,"N"): # verifica se o pinguim pode deslocar-se para Norte
                actions_list.append((p,"N"))
            if self.slide(state,x,y,"S"):
                actions_list.append((p,"S"))
            if self.slide(state,x,y,"E"):
                actions_list.append((p,"E"))
            if self.slide(state,x,y,"O"):
                actions_list.append((p,"O"))
        return sorted(actions_list) 
    

    def result (self, state, action):
        """
        Devolve um novo estado resultante de aplicar "action" a "state".
        """
        p,d = action # ação é um tuplo (ID pinguim, direção)
        pinguins = state.pinguins.copy()
        x,y = state.pinguins[p]
        (dx,dy) = self.directions[d]
        # desloca o pinguim até que embata num obstáculo ou noutro pinguim
        while (x+dx,y+dy) not in self.obstacles and (x+dx,y+dy) not in state.pinguins.values():
            x = x+dx
            y = y+dy
        # se o pinguim embateu num obstáculo, atualiza-se a posição do pinguim
        if (x+dx,y+dy) in self.obstacles:
            pinguins[p] = (x,y)
        # se o pinguim embateu noutro pinguim, ambos são removidos do estado
        if (x+dx,y+dy) in state.pinguins.values():
            pinguins.pop(p)
            for key in state.pinguins:
                if state.pinguins[key] == (x+dx,y+dy):
                    pinguins.pop(key)
                    break
        return EstadoPinguins(pinguins)
    

    def goal_test (self, state):
        """
        Verifica se todos os pinguins estão emparelhados, ou seja, se os estado está vazio.
        """
        return state.pinguins == {}
    

    def display (self, state):
        """
        Devolve a grelha em modo txt.
        """
        output = ""
        for i in range(self.dim[0]):
            for j in range(self.dim[1]):
                if (i,j) in state.pinguins.values():
                    for key in state.pinguins:
                        if state.pinguins[key] == (i,j):
                            ch = key
                            break
                elif (i,j) in self.obstacles:
                    ch = "##"
                elif (i,j) in self.water:
                    ch = "()"
                else:
                    ch = ".."
                output += ch + " "
            output += "\n"
        return output
    

    def executa(self, state, actions_list, verbose=False):
        """Executa uma sequência de acções a partir do estado devolvendo o triplo formado pelo estado, 
        pelo custo acumulado e pelo booleano que indica se o objectivo foi ou não atingido. Se o objectivo 
        for atingido antes da sequência ser atingida, devolve-se o estado e o custo corrente.
        Há o modo verboso e o não verboso, por defeito."""
        cost = 0
        for a in actions_list:
            seg = self.result(state,a)
            cost = self.path_cost(cost,state,a,seg)
            state = seg
            obj = self.goal_test(state)
            if verbose:
                print('Ação:', a)
                print(self.display(state),end='')
                print('Custo Total:',cost)
                print('Atingido o objectivo?', obj)
                print()
            if obj:
                break
        return (state, cost, obj)
    
    
    def halfPenguins(self, node):
        """
        Heurística que considera metade do número de pinguins.
        """
        return len(node.state.pinguins)//2
        
    def Npairings(self, node):
        """
        Heuristic that counts the number of direct pairings possible plus remaining unpaired penguins.
        Direct pairings are when two penguins are in the same row or column.
        Pairings are done in order of penguin IDs.
        """
        penguins = sorted(node.state.pinguins.items())  # Sort by penguin ID
        paired = set()
        np = 0  # Number of pairs formed
        
        for i, (p1, pos1) in enumerate(penguins):
            if p1 in paired:
                continue

            x1, y1 = pos1
            best_pair = None
            
            # # Find all possible pairings for current penguin
            # possible_pairs = []
            # for j in range(len(penguins)):
            #     if i == j:
            #         continue
            #     p2, pos2 = penguins[j]
            #     if p2 in paired:
            #         continue
            #     x2, y2 = pos2
            #     if x1 == x2 or y1 == y2:  # Same row or column
            #         possible_pairs.append(p2)
            
            # # Pair with the smallest ID possible penguin
            # if possible_pairs:
            #     p2 = min(possible_pairs)
            #     paired.add(p1)
            #     paired.add(p2)
            #     np += 1

            # Check all possible pairings for current penguin
            for j, (p2, pos2) in enumerate(penguins):
                if i == j or p2 in paired:
                    continue
                    
                x2, y2 = pos2
                
                # Check if in same row with clear path
                if x1 == x2 and self.clear_row(x1, y1, y2):
                    best_pair = p2
                    break
                    
                # Check if in same column with clear path
                if y1 == y2 and self.clear_col(x1, x2, y1):
                    best_pair = p2
                    break
            
            # If found a valid pair, mark both as paired
            if best_pair:
                paired.add(p1)
                paired.add(best_pair)
                np += 1
        
        nsp = len(node.state.pinguins) - 2 * np  # Number of single penguins
        return max(np + nsp, len(node.state.pinguins) // 2)
    
    def clear_row(self, x, y1, y2):
        """Check if horizontal path between y1 and y2 is clear"""
        step = 1 if y1 < y2 else -1
        for y in range(y1 + step, y2, step):
            if (x, y) in self.obstacles or (x, y) in self.water:
                return False
        return True

    def clear_col(self, x1, x2, y):
        """Check if vertical path between x1 and x2 is clear"""
        step = 1 if x1 < x2 else -1
        for x in range(x1 + step, x2, step):
            if (x, y) in self.obstacles or (x, y) in self.water:
                return False
        return True
    
    def highestPairings(self, node):
        """
        Heuristic that counts pairings by prioritizing penguins with most possible pairings.
        In case of tie, uses penguin ID.
        """
        penguin_pos = node.state.pinguins
        penguins = list(penguin_pos.keys())
        paired = set()
        np = 0  # Number of pairs formed
        
        while True:
            # Calculate possible pairs for each unpaired penguin
            pair_counts = {}
            possible_pairs = {}
            
            for p1 in penguins:
                if p1 in paired:
                    continue
                x1, y1 = penguin_pos[p1]
                possible = []
                for p2 in penguins:
                    if p1 == p2 or p2 in paired:
                        continue
                    x2, y2 = penguin_pos[p2]
                    if x1 == x2 or y1 == y2:
                        possible.append(p2)
                possible_pairs[p1] = possible
                pair_counts[p1] = len(possible)
            
            if not pair_counts:
                break
                
            # Find penguin with most possible pairings (smallest ID if tie)
            p1 = min(pair_counts.keys(), key=lambda p: (-pair_counts[p], p))
            
            if pair_counts[p1] == 0:
                break
                
            # Find best pair for p1 (with most possible pairings, then smallest ID)
            available_pairs = possible_pairs[p1]
            if available_pairs:
                p2 = min(available_pairs, key=lambda p: (-pair_counts.get(p, 0), p))
                paired.add(p1)
                paired.add(p2)
                np += 1
            else:
                break
        
        nsp = len(penguin_pos) - 2 * np  # Number of single penguins
        return np + nsp

In [2]:
p = PenguinsPairs()
print('Has to be: 2')
print('Got:', p.Npairings(Node(p.initial)))

Has to be: 2
Got: 2


In [3]:
line1 = "() () () () () () () () ()\n"
line2 = "## 02 00 .. .. .. .. 01 ##\n"
line3 = "## .. .. .. .. .. .. .. ##\n"
line4 = "## .. .. .. .. .. .. .. ##\n"
line5 = "## .. .. () () () .. .. ##\n"
line6 = "## .. .. .. .. .. .. 03 ##\n"
line7 = "## .. .. .. .. .. .. .. ##\n"
line8 = "## .. .. .. .. .. .. .. ##\n"
line9 = "() () () () () () () () ()\n"
grid2 = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8 + line9

p = PenguinsPairs(grid2)
print('Has to be: 3')
print('Got:', p.Npairings(Node(p.initial)))

Has to be: 3
Got: 3


In [4]:
line1 = "() () () () () () () () ()\n"
line2 = "## .. .. 07 .. 01 .. .. ##\n"
line3 = "## .. .. .. .. 05 .. .. ##\n"
line4 = "## .. .. .. 09 .. .. .. ##\n"
line5 = "## .. .. () () () 02 08 ##\n"
line6 = "## .. .. .. .. 04 .. .. ##\n"
line7 = "## .. 06 03 .. .. .. .. ##\n"
line8 = "## .. 00 .. .. .. .. .. ##\n"
line9 = "() () () () () () () () ()\n"
grid3 = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8 + line9

p = PenguinsPairs(grid3)
print('Has to be: 7')
print('Got:', p.Npairings(Node(p.initial)))

Has to be: 7
Got: 7


In [5]:
line1 = "() () () () () () () () ()\n"
line2 = "## .. .. 07 .. 01 .. .. ##\n"
line3 = "## .. .. .. .. 05 .. .. ##\n"
line4 = "## .. .. .. 09 .. .. .. ##\n"
line5 = "## .. .. () () () 02 08 ##\n"
line6 = "## .. .. .. .. 04 .. .. ##\n"
line7 = "## .. 06 03 .. .. .. .. ##\n"
line8 = "## .. 00 .. .. .. .. .. ##\n"
line9 = "() () () () () () () () ()\n"
grid3 = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8 + line9

print("Has to be:\n[('03', 'O'), ('02', 'E'), ('01', 'S'), ('09', 'O'), ('07', 'O'), ('07', 'S'), ('04', 'O'), ('00', 'O'), ('00', 'N')]\n\
expanded: 9")
print("Got:")
p = PenguinsPairs(grid3)
res,expanded = greedy_best_first_graph_search_plus_count(p,p.Npairings)
if res:
    print(res.solution())
else:
    print('No solution!')
print('expanded:',expanded)

Has to be:
[('03', 'O'), ('02', 'E'), ('01', 'S'), ('09', 'O'), ('07', 'O'), ('07', 'S'), ('04', 'O'), ('00', 'O'), ('00', 'N')]
expanded: 9
Got:
[('03', 'O'), ('02', 'E'), ('01', 'S'), ('09', 'O'), ('07', 'O'), ('07', 'S'), ('04', 'O'), ('00', 'O'), ('00', 'N')]
expanded: 9


In [6]:
line1 = "() () () () () () () () ()\n"
line2 = "## .. .. 07 .. 01 .. .. ##\n"
line3 = "## .. .. .. .. 05 .. .. ##\n"
line4 = "## .. .. .. 09 .. .. .. ##\n"
line5 = "## .. .. () () () 02 08 ##\n"
line6 = "## .. .. .. .. 04 .. .. ##\n"
line7 = "## .. 06 03 .. .. .. .. ##\n"
line8 = "## .. 00 .. .. .. .. .. ##\n"
line9 = "() () () () () () () () ()\n"
grid3 = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8 + line9

print("Has to be:\n[('04', 'E'), ('00', 'E'), ('09', 'O'), ('07', 'O'), ('07', 'S'), ('03', 'O'), ('02', 'E'), ('01', 'S'), ('00', 'N')]\n\
expanded: 673")
print("Got:")
p = PenguinsPairs(grid3)
res,expanded = astar_search_plus_count(p,p.Npairings)
if res:
    print(res.solution())
else:
    print('No solution!')
print('expanded:',expanded)

Has to be:
[('04', 'E'), ('00', 'E'), ('09', 'O'), ('07', 'O'), ('07', 'S'), ('03', 'O'), ('02', 'E'), ('01', 'S'), ('00', 'N')]
expanded: 673
Got:
[('04', 'E'), ('00', 'E'), ('09', 'O'), ('07', 'O'), ('07', 'S'), ('03', 'O'), ('02', 'E'), ('01', 'S'), ('00', 'N')]
expanded: 704


In [7]:
line1 = "() () () () () () () () ()\n"
line2 = "## 02 00 .. .. .. .. 01 ##\n"
line3 = "## .. .. .. .. .. .. .. ##\n"
line4 = "## .. .. .. .. .. .. .. ##\n"
line5 = "## .. .. () () () .. .. ##\n"
line6 = "## .. .. .. .. .. .. 03 ##\n"
line7 = "## .. .. .. .. .. .. .. ##\n"
line8 = "## .. .. .. .. .. .. .. ##\n"
line9 = "() () () () () () () () ()\n"
grid2 = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8 + line9

p = PenguinsPairs(grid2)
print('Has to be: 3')
print('Got:', p.highestPairings(Node(p.initial)))

Has to be: 3
Got: 3


In [8]:
line1 = "() () () () () () () () ()\n"
line2 = "## .. 00 .. .. .. .. .. ##\n"
line3 = "## .. 03 05 .. .. .. .. ##\n"
line4 = "## .. .. .. .. .. .. .. ##\n"
line5 = "## .. () .. () .. .. .. ##\n"
line6 = "## 04 .. 06 01 .. .. .. ##\n"
line7 = "## .. .. .. 02 .. .. .. ##\n"
line8 = "## .. 07 .. .. .. .. .. ##\n"
line9 = "() () () () () () () () ()\n"
grid4 = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8 + line9

p = PenguinsPairs(grid4)
print('Has to be: 6')
print('Got:', p.highestPairings(Node(p.initial)))

Has to be: 6
Got: 6


In [9]:
line1 = "() () () () () () () () ()\n"
line2 = "## .. .. 07 .. 01 .. .. ##\n"
line3 = "## .. .. .. .. 05 .. .. ##\n"
line4 = "## .. .. .. 09 .. .. .. ##\n"
line5 = "## .. .. () () () 02 08 ##\n"
line6 = "## .. .. .. .. 04 .. .. ##\n"
line7 = "## .. 06 03 .. .. .. .. ##\n"
line8 = "## .. 00 .. .. .. .. .. ##\n"
line9 = "() () () () () () () () ()\n"
grid3 = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8 + line9

p = PenguinsPairs(grid3)
print('Has to be: 7')
print('Got:', p.highestPairings(Node(p.initial)))

Has to be: 7
Got: 7


In [10]:
line1 = "() () () () () () () () ()\n"
line2 = "## .. 00 .. .. .. .. .. ##\n"
line3 = "## .. 03 05 .. .. .. .. ##\n"
line4 = "## .. .. .. .. .. .. .. ##\n"
line5 = "## .. () .. () .. .. .. ##\n"
line6 = "## 04 .. 06 01 .. .. .. ##\n"
line7 = "## .. .. .. 02 .. .. .. ##\n"
line8 = "## .. 07 .. .. .. .. .. ##\n"
line9 = "() () () () () () () () ()\n"
grid4 = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8 + line9

print("Has to be:\n[('05', 'S'), ('01', 'S'), ('07', 'O'), ('04', 'S'), ('00', 'S')]\n\
expanded: 5")
print("Got:")
p = PenguinsPairs(grid4)
res,expanded=astar_search_plus_count(p,p.highestPairings)
if res:
    print(res.solution())
else:
    print('No solution!')
print('expanded:',expanded)

Has to be:
[('05', 'S'), ('01', 'S'), ('07', 'O'), ('04', 'S'), ('00', 'S')]
expanded: 5
Got:
[('05', 'S'), ('01', 'S'), ('07', 'O'), ('04', 'S'), ('00', 'S')]
expanded: 5


In [11]:
line1 = "() () () () () () () () ()\n"
line2 = "## .. .. 07 .. 01 .. .. ##\n"
line3 = "## .. .. .. .. 05 .. .. ##\n"
line4 = "## .. .. .. 09 .. .. .. ##\n"
line5 = "## .. .. () () () 02 08 ##\n"
line6 = "## .. .. .. .. 04 .. .. ##\n"
line7 = "## .. 06 03 .. .. .. .. ##\n"
line8 = "## .. 00 .. .. .. .. .. ##\n"
line9 = "() () () () () () () () ()\n"
grid3 = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8 + line9

print("Has to be:\n[('04', 'E'), ('03', 'E'), ('03', 'N'), ('02', 'E'), ('09', 'O'), ('05', 'O'), ('05', 'S'), ('01', 'O'), ('00', 'N')]\n\
expanded: 577")
print("Got:")
p = PenguinsPairs(grid3)
res,expanded=astar_search_plus_count(p,p.highestPairings)
if res:
    print(res.solution())
else:
    print('No solution!')
print('expanded:',expanded)

Has to be:
[('04', 'E'), ('03', 'E'), ('03', 'N'), ('02', 'E'), ('09', 'O'), ('05', 'O'), ('05', 'S'), ('01', 'O'), ('00', 'N')]
expanded: 577
Got:
[('04', 'E'), ('03', 'E'), ('03', 'N'), ('01', 'O'), ('09', 'O'), ('05', 'O'), ('05', 'S'), ('02', 'E'), ('00', 'N')]
expanded: 777
